Ejercicio 2: Evaluación de la Función Pulmonar mediante
Regresión Múltiple

Descripción:
En este ejercicio, se trabajará con un conjunto de datos que simula la función
pulmonar de pacientes y su relación con variables fisiológicas y ambientales.
Se utilizará regresión múltiple para predecir la Capacidad Vital Forzada
(FVC), una medida clave en la evaluación de la salud pulmonar.

1. Carga y exploración de datos:
○ Genera un conjunto de datos con aproximadamente 500 registros,
siguiendo distribuciones fisiológicas realistas.
○ Identifica las variables numéricas y categóricas.
○ Realiza un análisis exploratorio con visualizaciones adecuadas.

In [7]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.preprocessing import StandardScaler, LabelEncoder, OneHotEncoder
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer

In [8]:

#Generación de datos
#Vamos a generar un conjunto de datos sintético con variables como la edad, 
# altura, peso, género, tabaquismo, exposición a contaminantes y 
# la variable dependiente FVC (Capacidad Vital Forzada).

# Establecemos una semilla para reproducibilidad
np.random.seed(42)

# Generamos datos
n = 500  # número de muestras

# Variables numéricas
edad = np.random.randint(18, 90, size=n)  # Edad entre 18 y 90 años
altura = np.random.normal(170, 10, size=n)  # Altura con media 170 cm y desviación 10
peso = np.random.normal(70, 15, size=n)  # Peso con media 70 kg y desviación 15

# Variables categóricas
genero = np.random.choice(['Masculino', 'Femenino'], size=n)  # Género
tabaquismo = np.random.choice([0, 1], size=n)  # 0: no fumador, 1: fumador
exposicion_contaminantes = np.random.choice([0, 1], size=n)  # 0: baja exposición, 1: alta exposición




# Variable objetivo: FVC (Capacidad Vital Forzada)
fvc = np.random.normal(2.5, 0.5, size=n)  # FVC con una media de 2.5 y desviación estándar de 0.5


# Creando el DataFrame
df = pd.DataFrame({
    'Edad': edad,
    'Altura': altura,
    'Peso': peso,
    'Género': genero,
    'Tabaquismo': tabaquismo,
    'Exposición_contaminantes': exposicion_contaminantes,
    'FVC': fvc
})

# Mostrar las primeras filas del DataFrame
df.head()

,Edad,Altura,Peso,Género,Tabaquismo,Exposición_contaminantes,FVC
0,69,162.815558,60.433900,Masculino,1,0,2.688550
1,32,167.865528,62.035046,Masculino,1,0,2.485369
2,89,173.109076,60.652892,Femenino,1,1,3.063025
3,78,184.753562,61.667843,Masculino,1,1,2.474303
4,38,178.576596,60.439193,Femenino,0,0,1.613484


In [9]:
##Análisis exploratorio de datos (EDA)
#Este paso incluye el análisis visual y la comprensión de la distribución de las variables.

# Visualización de las correlaciones entre las variables numéricas
plt.figure(figsize=(10, 6))
sns.heatmap(df.corr(), annot=True, cmap='coolwarm', fmt='.2f')
plt.title('Matriz de Correlación')
plt.show()

# Distribución de la FVC
plt.figure(figsize=(8, 5))
sns.histplot(df['FVC'], kde=True, color='blue')
plt.title('Distribución de la Capacidad Vital Forzada (FVC)')
plt.show()

# Boxplots de variables categóricas vs FVC
plt.figure(figsize=(8, 5))
sns.boxplot(x='Género', y='FVC', data=df)
plt.title('FVC vs Género')
plt.show()

plt.figure(figsize=(8, 5))
sns.boxplot(x='Tabaquismo', y='FVC', data=df)
plt.title('FVC vs Tabaquismo')
plt.show()

ValueError: could not convert string to float: 'Masculino'

<Figure size 1000x600 with 0 Axes>

2. Preprocesamiento:
○ Normaliza las variables numéricas.
○ Codifica las variables categóricas utilizando OneHotEncoder.
○ Separa la variable objetivo (FVC) de las características
predictoras.
○ Divide los datos en entrenamiento y prueba (por ejemplo, 75%-25%).

In [ ]:
#Preprocesamiento de los datos
#Vamos a normalizar las variables numéricas, 
# codificar las variables categóricas utilizando OneHotEncoder, 
# separar las características predictoras de la variable objetivo y 
# dividir los datos en conjuntos de entrenamiento y prueba.

# Variables predictoras y objetivo
X = df.drop('FVC', axis=1)  # Variables predictoras
y = df['FVC']  # Variable objetivo

# Dividir los datos en entrenamiento (75%) y prueba (25%)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

# Columnas numéricas y categóricas
num_features = ['Edad', 'Altura', 'Peso']
cat_features = ['Género', 'Tabaquismo', 'Exposición_contaminantes']

# Preprocesamiento de datos
# Normalización de variables numéricas y codificación OneHot para variables categóricas
preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), num_features),  # Normalizar las variables numéricas
        ('cat', OneHotEncoder(drop='first'), cat_features)  # Codificar las variables categóricas
    ])

# Crear el pipeline con el preprocesador y el modelo de regresión lineal
pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),  # Preprocesamiento de datos
    ('regressor', LinearRegression())  # Regresión lineal